In [1]:
import sys
sys.path.append("/home/evlasova/mirpy")

In [2]:
import os

os.chdir('../')

In [4]:
from mir.common.parser import *
from mir.common.repertoire import Repertoire
from mir.common.repertoire_dataset import RepertoireDataset
from mir.common.clonotype_dataset import ClonotypeDataset
from mir.basic.segment_usage import *
from mir.basic.sampling import RepertoireSampling
from mir.biomarkers.fisher_biomarkers_detector import FisherBiomarkersDetector
from mir.comparative.pair_matcher import ClonotypeRepresentation
from mir.basic.pgen import OlgaModel
import time

In [5]:
metadata_beta = pd.read_csv('data/desc_fmba_new_split_beta.csv').drop(columns=['Unnamed: 0']).dropna()

In [6]:
metadata_beta

,run,COVID_status,covid,folder,is_test_run
0,050002290808_S117_L002,COVID,covid,#2,False
1,050002630808_S113_L002,COVID,covid,#2,False
2,050003120808_S112_L002,COVID,covid,#2,False
3,050003130808_S114_L002,COVID,covid,#2,False
4,050003450808_S118_L002,COVID,covid,#2,False
...,...,...,...,...,...
1220,p18_72_DNA_S88_L002,COVID,covid,#9,False
1221,p18_73_DNA_S89_L002,COVID,covid,#9,False
1222,p18_74_DNA_S90_L002,COVID,covid,#9,False
1223,p18_75_DNA_S91_L002,COVID,covid,#9,False


In [7]:
metadata_alpha = pd.read_csv('data/desc_fmba_new_split_alpha.csv').drop(
    columns=['Unnamed: 0']).dropna().reset_index(drop=True)
metadata_alpha

,run,COVID_status,covid,folder,is_test_run
0,050002290808_S117_L002,COVID,covid,#2,False
1,050002630808_S113_L002,COVID,covid,#2,False
2,050003120808_S112_L002,COVID,covid,#2,False
3,050003130808_S114_L002,COVID,covid,#2,False
4,050003450808_S118_L002,COVID,covid,#2,False
...,...,...,...,...,...
1219,p18_72_DNA_S88_L002,COVID,covid,#9,False
1220,p18_73_DNA_S89_L002,COVID,covid,#9,False
1221,p18_74_DNA_S90_L002,COVID,covid,#9,False
1222,p18_75_DNA_S91_L002,COVID,covid,#9,False


In [8]:
markers = pd.read_csv('figures/covid_clonotypes.csv')
markers

,cdr3,cluster,has_covid_association,chain,v,j
0,CASAPGGSYEQYF,0,False,beta,TRBV12-5/TRBV7-9/TRBV28,TRBJ2-7
1,CASIPGGSYEQYF,0,False,beta,TRBV6-5,TRBJ2-7
2,CASKPGGSYEQYF,0,False,beta,TRBV5-4/TRBV25-1/TRBV6-5,TRBJ2-7
3,CASLPGGSYEQYF,0,False,beta,TRBV28,TRBJ2-7
4,CASNPGGSYEQYF,0,False,beta,TRBV28,TRBJ2-7
...,...,...,...,...,...,...
4955,CVVSDLMEYGNKLVF,111,False,alpha,TRAV8-2,TRAJ47
4956,CVVSDRMEYGNKLVF,111,False,alpha,TRAV8-2,TRAJ47
4957,CVVSEPSGGYQKVTF,112,False,alpha,TRAV8-2/TRAV10,TRAJ13
4958,CVVSGYGAAGNKLTF,113,False,alpha,TRAV8-2/TRAV10,TRAJ17


In [9]:
beta_markers = markers[markers.chain == 'beta']
alpha_markers = markers[markers.chain == 'alpha']

In [ ]:
beta_markers

In [ ]:
alpha_markers

In [ ]:
beta_representations = [ClonotypeRepresentation(cdr3aa=x[1].cdr3, 
                                                             v=x[1].v, 
                                                             j=x[1].j) for x in beta_markers.iterrows()]

In [ ]:
alpha_representations = [ClonotypeRepresentation(cdr3aa=x[1].cdr3, 
                                                             v=x[1].v, 
                                                             j=x[1].j) for x in alpha_markers.iterrows()]

In [ ]:
t0 = time.time()
initial_dataset_TRB = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=metadata_beta,
                                 threads=32,
                                 paths=[f'/projects/fmba_covid/publication/fmba/{r["run"]}.clonotypes.TRB.txt' for _, r in metadata_beta.iterrows()])
print(time.time() - t0)

In [ ]:
initial_dataset_TRB.clonotype_usage_matrix

In [ ]:
len(initial_dataset_TRB.clonotype_usage_matrix.public_clonotypes)

In [ ]:
healthy_dataset, ill_dataset = initial_dataset_TRB.split_by_metadata_function(lambda x: x.COVID_status == 'healthy')

In [ ]:
healthy_dataset

In [ ]:
ill_dataset

In [ ]:
# healthy_dataset.threads = 2

In [ ]:
healthy_dataset.clonotype_usage_matrix.clonotype_database_usage

In [ ]:
# ill_dataset.threads = 2

In [ ]:
ill_dataset.clonotype_usage_matrix.clonotype_database_usage

In [ ]:
fbd = FisherBiomarkersDetector(control_repertoire_dataset=healthy_dataset,
                               ill_repertoire_dataset=ill_dataset, 
                               adjusted_p_value=0.05, 
                               threads=16)

In [ ]:
t0 = time.time()
biomarkers = fbd.detect_biomarkers(adjusted_p_value=0.05)
print(time.time() - t0)

In [ ]:
len(biomarkers)

In [ ]:
cd = ClonotypeDataset(clonotypes=[ClonotypeAA(cdr3aa=x.cdr3aa) for x in biomarkers])

In [ ]:
cd.clonotype_clustering

In [ ]:
fig, ax = plt.subplots()
cd.plot_clonotype_clustering(color_by='cluster', ax=ax, plot_unclustered=True)
ax.legend([])

In [ ]:
initial_dataset_TRB.metadata

In [ ]:
import random

In [ ]:
r_dataset_1, r_dataset_2 = initial_dataset_TRB.split_by_metadata_function(lambda x: bool(random.choice([0, 1, 1])))

In [ ]:
r_dataset_1.threads = 2

In [ ]:
r_dataset_2.threads = 2

In [ ]:
r_dataset_1.clonotype_usage_matrix.clonotype_database_usage

In [ ]:
r_dataset_2.clonotype_usage_matrix.clonotype_database_usage

In [ ]:
fbdr1 = FisherBiomarkersDetector(control_repertoire_dataset=r_dataset_1,
                               ill_repertoire_dataset=r_dataset_2, 
                               adjusted_p_value=0.05, 
                               threads=16)

In [ ]:
fbdr2 = FisherBiomarkersDetector(control_repertoire_dataset=r_dataset_2,
                               ill_repertoire_dataset=r_dataset_1, 
                               adjusted_p_value=0.05, 
                               threads=16)

In [ ]:
biomarkers_1_2 = fbdr1.detect_biomarkers()

In [ ]:
biomarkers_2_1 = fbdr2.detect_biomarkers()

In [ ]:
biomarkers_1_2

In [ ]:
biomarkers_2_1